In [10]:
from huggingface_hub import notebook_login

In [13]:
# 需要在huggingface官网绑定相关的token进行认证
notebook_login()

In [15]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "zh-CN", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "zh-CN", split="test", use_auth_token=True)

print(common_voice)

KeyboardInterrupt: 